In [1]:
import os
import shutil
import pickle
import logging
from pathlib import Path
import configparser

# class EnvironmentManager:
#     _environments = {}  # Stores Environment instances keyed by environment_name

#     def __init__(self, config_file):
#         self.config = configparser.ConfigParser()
#         self.config.read(config_file)

#         self.DIR_STRUCTURE = self.config._sections['DIR_STRUCTURE']
#         self.GHPATH = self.config.get('PATHS', 'GHPATH')
#         self.VANILLA_PATH = self.config.get('PATHS', 'VANILLA_PATH')
#         self.GH_FILE_PATH = self.config.get('PATHS', 'GH_FILE_PATH')

#     class Environment:
#         def __init__(self, environment_name, config):
#             self.environment_name = environment_name
#             self.base_path = Path(config.get('PATHS', 'BASE_PATH')) / f"{environment_name}"
#             self.base_path.mkdir(parents=True, exist_ok=True)
            
#             self.dirs = {name: self.base_path / dirname for name, dirname in config._sections['DIR_STRUCTURE'].items()}
#             new_dirs = {}
#             for path in list(self.dirs.values()):
#                 path.mkdir(parents=True, exist_ok=True)
#                 if path.name == "03-GH_Files":
#                     raw = "03a-Raw"
#                     raw_path = path / raw
#                     raw_path.mkdir(parents=True, exist_ok=True)
#                     new_dirs["raw"] = raw_path
#                     processed = "03b-Processed"
#                     processed_path = path / processed
#                     processed_path.mkdir(parents=True, exist_ok=True)
#                     new_dirs["processed"] = processed_path

#             self.dirs.update(new_dirs)

#             self.gh_path = Path(config.get('PATHS', 'GHPATH'))
#             self.initialise(config)

#         def serialize(self):
#             file_path = self.base_path / "environment.pkl"
#             with open(file_path, "wb") as file:
#                 pickle.dump(self, file)

#         def clone_env(self, clone=False):
#             GH_path = Path(self.gh_path)
#             lib_path = self.dirs.get('ghlib')
#             r, d, f = next(os.walk(lib_path))
#             if clone and len(f) == 0:
#                 for root, dirs, files in os.walk(GH_path):
#                     root_path = Path(root)
#                     relative_path = root_path.relative_to(GH_path)
#                     for file in files:
#                         source_path = root_path / file
#                         destination_path = lib_path / relative_path / file
#                         destination_path.parent.mkdir(parents=True, exist_ok=True)
#                         shutil.copy(source_path, destination_path)

#         def reinstate_env(self, override=False):
#             if input("Are you sure you want to reinstate the environment? \n"
#                      "This will replace existing files in GH Libraries Folder"
#                      "with those belonging to this environment? [y/n]").lower() == "y" \
#                     or override:
#                 print("Reinstating environment...")
#                 GH_path = self.gh_path
#                 lib_path = self.dirs["ghlib"]

#                 for root, dirs, files in os.walk(GH_path):
#                     for file in files:
#                         file_path = Path(root) / file
#                         if file_path.is_file():
#                             file_path.unlink()

#                 for file in lib_path.iterdir():
#                     if file.is_file():
#                         shutil.copy(file, GH_path)
#             else:
#                 print("Reinstate cancelled")

#         def view_gh_environment(self):
#             for root, dirs, files in os.walk(self.dirs["ghlib"]):
#                 for file in files:
#                     print(file)

#         def copy_file(self, src, dst):
#             if not src.exists() or not src.is_file():
#                 print(f"Source file does not exist or is not a file: {src}")
#                 return

#             dst.parent.mkdir(parents=True, exist_ok=True)

#             try:
#                 shutil.copyfile(src, dst)
#                 print(f"File copied successfully from {src} to {dst}.")
#             except Exception as e:
#                 print(f"Error copying file from {src} to {dst}: {e}")

#         def initialise(self, config, vanilla_components: Path = None, clone_env=True):
#             print("Setting environment variables")
#             print("Copying vanilla components")
#             if vanilla_components is None:
#                 vanilla_components = Path(config.get('PATHS', 'VANILLA_PATH'))
#             if vanilla_components:
#                 vanilla_components_destination = self.dirs["vanilla"]
#                 vanilla_components_destination.mkdir(parents=True, exist_ok=True)
#                 destination_path = vanilla_components_destination / vanilla_components.name
#                 self.copy_file(vanilla_components, destination_path)
#             print("Copying components")
#             self.clone_env(clone_env)
#             print("Copying gh files")
#             self.copy_ghtest_files(config)

#         def copy_ghtest_files(self, config):
#             try:
#                 for root, dirs, files in os.walk(config.get('PATHS', 'GH_FILE_PATH')):
#                     for file in files:
#                         if file.endswith(".gh"):
#                             shutil.copy(os.path.join(root, file), self.dirs["raw"])
#             except PermissionError as e:
#                 print(f"Permission error on {e}")

#         def get_gh_file(self):
#             for root, dirs, files in os.walk(self.dirs["files"]):
#                 for file in files:
#                     if file.endswith(".gh"):
#                         yield os.path.join(root, file)
#             print("No more gh files")

#     def create_environment(self, environment_name):
#         if environment_name not in self._environments:
#             self._environments[environment_name] = self.Environment(environment_name, self.config)
#         return self._environments[environment_name]

#     def get_directory_names(self):
#         return list(self.DIR_STRUCTURE.values())

#     def get_environment(self):
#         return list(self._environments.values())[0]

#     def setup_logging(self, environment_name):
#         log_directory = self._environments[environment_name].dirs['logs']
#         log_file_path = log_directory / "GH.log"
#         print(log_file_path)
#         logging.basicConfig(level=logging.DEBUG,
#                             format='%(asctime)s - %(name)s - %(levelname)s: %(message)s',
#                             filemode="w",
#                             filename=str(log_file_path))

In [6]:
from gh_extraction import *

class GHComponentProxy:
    """
    A class that represents an uninstantiated node component. This class is used to
    interface with the Grasshopper environment and manage component metadata.
    """
    component_server = Grasshopper.Instances.ComponentServer

    def __init__(self, obj_proxy: ghk.IGH_ObjectProxy):
        self.obj_proxy = obj_proxy
        self.category = str(obj_proxy.Desc.Category)
        self.name = str(obj_proxy.Desc.Name)
        self.guid = str(obj_proxy.Guid)
        self.nickname = str(obj_proxy.Desc.NickName)
        self.description = str(obj_proxy.Desc.Description)
        self.type = str(obj_proxy.Type)
        self.library = self.get_assembly_name()
        self.inputs, self.outputs = self.process_parameters()

    @property
    def sys_guid(self):
        return System.Guid(self.guid)

    @property
    def lib_guid(self):
        return self.obj_proxy.LibraryGuid

    def to_dict(self):
        return {
            "category": self.category,
            "name": self.name,
            "guid": self.guid,
            "nickname": self.nickname,
            "description": self.description[:60].replace("\n", " ").replace("\r", " ").strip(),
            "type": self.type,
            "library": self.library,
            'paramin': self.inputs,
            'paramout':self.outputs
        }

    def get_assembly(self) -> System.Reflection.Assembly:
        """
        Returns the AssemblyInfo object of a component. This method interfaces with
        the Grasshopper component server to retrieve assembly information.
        """
        return GHComponentProxy.component_server.FindAssembly(self.lib_guid)

    def get_assembly_name(self):
        """
        Retrieves the name of the assembly containing the component. This is used to
        identify the source library of the component.
        """
        assembly = self.get_assembly()
        return assembly.Name if assembly else "Unknown"

    def process_parameters(self):
        comp_data = {'inputs': [], 'outputs': []}
        prox = self.obj_proxy
        name = prox.Desc.Name
        p = prox.CreateInstance()
        try:
            p = IGH_Component(p)
            params = [param for param in p.Params.GetEnumerator()]
            kinds = [int(prm.Kind) for prm in params]
            typnames = [prm.TypeName for prm in params]
            param_properties = list(zip(typnames, kinds))
            for t, n in param_properties:
                if n == 2:
                    comp_data['inputs'].append(t)
                elif n == 3:
                    comp_data['outputs'].append(t)
        except Exception as e:
            try:
                p = IGH_Param(p)
                k = p.TypeName
                comp_data['inputs'].append(k) # Assuming this was intended to be managed differently.
                comp_data['outputs'].append(k) 
            except Exception as e:
                pass
        return comp_data["inputs"], comp_data["outputs"]

    def __str__(self):
        return f"{self.category}:{self.name}"

    def __repr__(self):
        return self.__str__()


class GHComponentTable:
    cs = Grasshopper.Instances.ComponentServer.ObjectProxies
    compserver = Grasshopper.Instances.ComponentServer
    component_dict = {component.Guid: component for component in cs}
    vanilla_proxies = {}
    object_proxies = []
    non_native_proxies = []
    _guid_to_idx = {}
    _idx_to_guid = {}
    df = None

    @classmethod
    def initialise(cls, environment_manager, vanilla_components_location=None):
        if vanilla_components_location is None:
            vanilla_components_location = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240318-VanillaComponents"
        cls.vanilla_proxies = {obj.sys_guid: obj for obj in cls.load_vanilla_gh_proxies(vanilla_components_location)}
        cls.object_proxies = [GHComponentProxy(obj) for obj in cls.cs]
        cls.non_native_proxies = sorted(
            [ghp for ghp in cls.object_proxies if not cls.is_native(ghp)],
            key=lambda x: str(x))
        cls.object_proxies = list(cls.vanilla_proxies.values()) + cls.non_native_proxies
        env = environment_manager.get_environment()
        df_directory = env.dirs['all']
        cls.obj_lookup = {obj.sys_guid: obj for obj in cls.object_proxies}
        cls.df = cls.table_to_pandas(df_directory, 'all_components')
        cls.to_csv()

    @classmethod
    def to_csv(cls, location=None, name="grasshopper_components.csv"):
        if location is None:
            location = r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240318-AllComponents"
        filename = Path(location) / name
        if not filename.exists():
            with open(filename, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                if cls.object_proxies:
                    header = cls.object_proxies[0].to_dict().keys()
                    writer.writerow(header)
                    for proxy in cls.object_proxies:
                        writer.writerow(proxy.to_dict().values())


    @classmethod
    def export_vanilla_gh_table(cls, filepath):
        datetime_str = datetime.datetime.now().strftime("%y%m%d")
        filename = Path(filepath) / str(str(datetime_str) + " vanilla_components.csv")
        if not filename.exists():
            with open(filename, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                if cls.object_proxies:
                    header = cls.object_proxies[0].to_dict().keys()
                    writer.writerow(header)
                    for proxy in cls.object_proxies:
                        writer.writerow(proxy.to_dict().values())
                        print(f"Exported vanilla components to CSV: {filename}")
                else:
                    cls.object_proxies = sorted([GHComponentProxy(obj) for obj in cls.cs], key=lambda x: x.type)
                    with open(filename, mode='w', newline='', encoding='utf-8') as file:
                        writer = csv.writer(file)
                        if cls.object_proxies:
                            header = cls.object_proxies[0].to_dict().keys()
                            writer.writerow(header)
                            for proxy in cls.object_proxies:
                                writer.writerow(proxy.to_dict().values())
                            print(f"Exported vanilla components to CSV: {filename}")


    @classmethod
    def search_component_by_guid(cls, guid: System.Guid):
        return cls.component_dict[guid]

    @classmethod
    def is_native(cls, ghp):
        return cls.vanilla_proxies.get(ghp.sys_guid, False)

    @classmethod
    def convert_csv_line_to_proxies(cls, guid_str: str):
        guid = System.Guid(guid_str)
        proxy_object = cls.search_component_by_guid(guid)
        if proxy_object:
            logging.debug(f"Proxy object: {proxy_object.Desc.Name} found")
            return GHComponentProxy(proxy_object)
        logging.warning(f"Could not find a vanilla GH with guid {guid_str}")
        return None

    @classmethod
    def load_vanilla_gh_proxies(cls, filepath, file='vanilla_components.csv'):
        vanilla_proxies = []
        location = Path(filepath) / file
        if location.exists():
            with open(location, mode='r') as cc:
                reader = csv.DictReader(cc)
                for row in reader:
                    guid_str = row['guid']
                    gh_proxy = cls.convert_csv_line_to_proxies(guid_str)
                    if gh_proxy:
                        vanilla_proxies.append(gh_proxy)
            return vanilla_proxies
        logging.warning(f"Could not find a vanilla components file: {filepath}")

    @classmethod
    def view_all_categories(cls):
        categories = set([pr.category for pr in cls.object_proxies])
        return categories

    @classmethod
    def view_proxies(cls, n):
        print(f"There are {len(cls.object_proxies)} proxies")
        for proxy in cls.object_proxies[:n]:
            print(proxy)

    @classmethod
    def table_to_pandas(cls, location, filename='all_components.csv'):
        full_path = Path(location) / filename

        # Ensure the CSV exists, if not, create it
        if not full_path.exists():
            cls.to_csv(location, filename)  # Make sure to pass both location and filename

        # Now, the file should exist. Read it into a pandas DataFrame
        df = pd.read_csv(full_path)  # Use full_path, which is the complete path to the file

        # Building the GUID to index mappings
        cls._guid_to_idx = {row['guid']: idx for idx, row in df.iterrows()}
        cls._idx_to_guid = {idx: row['guid'] for idx, row in df.iterrows()}
        return df


    @classmethod
    def guid_to_component_name(cls, guid, category=True):
        if type(guid) != System.Guid:
            guid = System.Guid(guid)
        proxy = cls.component_dict.get(guid)
        if category:
            return f"{proxy.Desc.Category}:{proxy.Desc.Name}"
        return f":{proxy.Desc.Name}"
    @classmethod
    def get_guid_to_idx(cls, guid: System.Guid) -> int:
        return cls._guid_to_idx.get(str(guid), None)

    @classmethod
    def get_idx_to_guid(cls, idx: int) -> System.Guid:
        guid_str = cls._idx_to_guid.get(idx, None)
        return System.Guid(guid_str) if guid_str else None

    @classmethod
    def component_to_idx(cls, component) -> int:
        component_category = component.category

        component_name = component.name
        if cls.df is not None:
            id = str(component.obj.ComponentGuid)
            q = cls.df.query("`guid` == @id")
            if q.empty:
                raise ValueError(f"Component {component_name} not found in dataframe.")
            else:
                return q.index[0]
        else:
            raise ValueError(f"Cant Search an empty dataframe.")

    @classmethod
    def idx_to_component(cls, idx):
        return cls.df["nickname"].iloc[idx]

    @classmethod
    def idx_to_component2(cls, idx):
        return cls.df["name"].iloc[idx]


    @classmethod
    def get_guid_by_type(cls, type_search):
        # this is used when preprocessing a file and returns the component id given the object type
        matches = cls.df[cls.df['type'] == type_search]
        if not matches.empty:
            return matches['guid'].tolist()
        else:
            return []

ModuleNotFoundError: No module named 'gh_extraction'

In [3]:
env_manager = EnvironmentManager(config_file='config.ini')
env = env_manager.create_environment('testenbv')
GHComponentTable.initialise(env_manager)

Setting environment variables
Copying vanilla components
File copied successfully from C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240318-VanillaComponents\vanilla_components.csv to C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Environments\testenbv\00-VanillaComponents\vanilla_components.csv.
Copying components
Copying gh files


In [5]:
GHComponentTable.df.head()

,category,name,guid,nickname,description,type,library,paramin,paramout
0,X,Example,d541404d-b806-453e-8351-a954e824ed22,X,Example,Grasshopper.Kernel.Components.GH_DocExampleCom...,Grasshopper,"['Circle', 'Line']","['Arc', 'Arc']"
1,Curve,Center,afff17ed-5975-460b-9883-525ae0677088,Cen,Find the center point and radius of arcs and c...,AnalysisComponents.Component_ArcCenterPoint,Curve Components,['Arc'],"['Point', 'Number']"
2,Curve,Evaluate Length,6b021f56-b194-4210-b9a1-6cef3b7d0848,Eval,Evaluate a curve at a certain factor along its...,AnalysisComponents.Component_ArcLengthPoint,Curve Components,"['Curve', 'Number', 'Boolean']","['Point', 'Vector', 'Number']"
3,Curve,Closed,323f3245-af49-4489-8677-7a2c73664077,Cls,Test if a curve is closed or periodic.,AnalysisComponents.Component_Closed,Curve Components,['Curve'],"['Boolean', 'Boolean']"
4,Curve,Closed [OBSOLETE],f2030fa9-db3f-437e-9b50-5607db6daf87,Cls,This component is OBSOLETE. It has been replac...,AnalysisComponents.Component_Closed_OBSOLETE,Curve Components,['Curve'],['Boolean']
